# Notebook to scrape MI High School Baseball school and conference info to integrate with the field measurements

In [ ]:
## Dependencies: requests, bs4

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen

In [ ]:
## Web Addresses

top_page_url = 'https://en.wikipedia.org/wiki/List_of_Michigan_High_School_Association_member_conferences'

top_html = urlopen(top_page_url)

soup = BeautifulSoup(top_html, 'html.parser')

## Wednesday 12-15 work below


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### test scrape of the CAAC conference

### download the page

test_url = 'https://en.wikipedia.org/wiki/Capital_Area_Activities_Conference'

soup = (BeautifulSoup(test_url, 'html.parser'))

table_class='wikitable sortable jquery-tablesorter'

response = requests.get(test_url)
print(response.status_code)


200


In [6]:
#### Parse the data from html

soup = BeautifulSoup(response.text, 'html.parser')
table_1 = soup.find('table',{'class':'wikitable'})
table_1

<table class="wikitable sortable">
<tbody><tr>
<th>Team
</th>
<th>Class
</th>
<th>Enrollment
</th>
<th>Joined
</th>
<th>City
</th>
<th>County
</th>
<th>Previous Conference
</th></tr>
<tr>
<th colspan="7" style="background:#0000FF; color:white">Blue Division
</th></tr>
<tr>
<td><a href="/wiki/DeWitt_High_School_(Michigan)" title="DeWitt High School (Michigan)">DeWitt Panthers</a>
</td>
<td>A
</td>
<td>1013
</td>
<td>2003
</td>
<td>DeWitt
</td>
<td>Clinton
</td>
<td>Ingham County League
</td></tr>
<tr>
<td><a href="/wiki/East_Lansing_High_School" title="East Lansing High School">East Lansing Trojans</a>
</td>
<td>A
</td>
<td>1119
</td>
<td>2003
</td>
<td>East Lansing
</td>
<td>Ingham
</td>
<td>Capital Area Conference
</td></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/Grand_Ledge_Public_Schools" title="Grand Ledge Public Schools">Grand Ledge Comets</a>
</td>
<td>A
</td>
<td>1687
</td>
<td>2003
</td>
<td>Grand Ledge
</td>
<td>Eaton
</td>
<td>Capital Area Conference
</td></tr>
<tr>
<td>

In [17]:
### read the html into a dataframe

df = pd.read_html(str(table_1))
blue_df = pd.DataFrame(df[0])
blue_df.info()

## Output the data to a csv file
blue_df.to_csv('TEMP/blue.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   (Team, Blue Division)                 21 non-null     object
 1   (Class, Blue Division)                21 non-null     object
 2   (Enrollment, Blue Division)           21 non-null     object
 3   (Joined, Blue Division)               21 non-null     object
 4   (City, Blue Division)                 21 non-null     object
 5   (County, Blue Division)               21 non-null     object
 6   (Previous Conference, Blue Division)  21 non-null     object
dtypes: object(7)
memory usage: 1.3+ KB


In [23]:
### Test scrape the Catholic High School League

test_url = 'https://en.wikipedia.org/wiki/Catholic_High_School_League'

response = requests.get(test_url)
soup = BeautifulSoup(response.text, 'html.parser')


table = soup.find('table',{'class':'wikitable'})

df = pd.read_html(str(table))
catholic_df = pd.DataFrame(df[0])
catholic_df.info()

## Output to check data
catholic_df.to_csv('TEMP/catholic.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 8 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   (Catholic High School League, School)         32 non-null     object
 1   (Catholic High School League, Location)       32 non-null     object
 2   (Catholic High School League, Enrollment[1])  32 non-null     int64 
 3   (Catholic High School League, Class)          32 non-null     object
 4   (Catholic High School League, Nickname)       32 non-null     object
 5   (Catholic High School League, Colors)         32 non-null     object
 6   (Catholic High School League, Gender)         32 non-null     object
 7   (Catholic High School League, Established)    32 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
